# compile posterior for the BGS catalogs

In [1]:
import os
import h5py
import numpy as np
import astropy.table as aTable

In [2]:
import healpy


Bad key text.latex.preview in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95 

## compile BGS Bright

In [3]:
dir_dat = '/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/'

In [4]:
_bgs_n = aTable.Table.read(os.path.join(dir_dat, 'BGS_BRIGHT_N_clustering.dat.fits'))
_bgs_s = aTable.Table.read(os.path.join(dir_dat, 'BGS_BRIGHT_S_clustering.dat.fits'))

bgs_bright = aTable.vstack([_bgs_n, _bgs_s])

In [5]:
len(bgs_bright)

143853

In [6]:
bgs_bright[:5]

RA,DEC,TARGETID,NTILE,TILES,Z,COMP_TILE,rosette_number,rosette_r,FRACZ_TILELOCID,BITWEIGHTS [2],PROB_OBS,WEIGHT_ZFAIL,WEIGHT,flux_g_dered,flux_r_dered,flux_z_dered,flux_w1_dered,flux_w2_dered,NZ,WEIGHT_FKP
float64,float64,int64,int64,bytes43,float64,float64,float64,float64,float64,int64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float64
191.20458196461243,60.31535787110577,39633390185481345,1,100,0.08969256109731599,0.64,3.0,1.613220114177946,1.0,-1 .. -1,1.0,1.0,1.0,11.618794,18.39226,20.892622,13.421043,9.27371,0.04778151135868713,0.0030821516772042766
187.04622815471808,60.82670180284912,39633396124617897,1,100,0.37400455377945924,0.64,3.0,1.6801915534232357,1.0,-1 .. -1,1.0,1.0,1.0,4.046807,16.82531,38.79777,85.02852,54.11833,0.0009577827729000268,0.133626516949366
187.83760708675698,60.38987543677365,39633393155049303,1,100,0.1731108191825362,0.64,3.0,1.7267282368438097,0.5,6932875859169517021 .. -6858338461939850770,0.4609375,1.0,2.15,13.26433,27.728409,46.020153,37.03337,25.867756,0.01270745015908088,0.01149149450827468
187.05117249536465,62.614542270156655,39633416324383107,1,100,0.2703579310142208,0.64,3.0,1.5579854421783794,1.0,-1 .. -1,1.0,1.0,1.0,4.4575434,17.798513,37.420948,46.075268,30.963612,0.004051015958871262,0.035183206585599834
191.14757099005186,60.345815959235,39633390185480938,1,100,0.2819459922463374,0.64,3.0,1.5743490921284766,0.5,4864741233190265570 .. 434972757299228716,0.359375,1.0,2.74468085106383,6.782468,16.732075,29.996437,31.939342,25.343126,0.004051015958871262,0.035183206585599834


In [7]:
bgs_bright['phi'] = np.radians(bgs_bright['RA'])
bgs_bright['theta'] = np.radians(90. - bgs_bright['DEC'])
bgs_bright['healpix'] = healpy.ang2pix(64, bgs_bright['theta'], bgs_bright['phi'], nest=True)

get all healpixs

In [8]:
tiles_fuji = aTable.Table.read('/global/cfs/cdirs/desi/spectro/redux/fuji/healpix/tilepix.fits')
is_bright = (tiles_fuji['PROGRAM'] == 'bright')
is_sv3 = (tiles_fuji['SURVEY'] == 'sv3')

hpixs = np.unique(np.sort(np.array(tiles_fuji['HEALPIX'][is_bright & is_sv3])))

In [9]:
mstars = np.tile(-999., (len(bgs_bright), 30))

for hpix in hpixs: 
    try: 
        f = h5py.File('/global/cfs/cdirs/desi/users/chahah/provabgs/svda/provabgs-sv3-bright-%s.BGS_BRIGHT.hdf5' % hpix, 'r')
        tid = f['targetid'][...]
        mcmc = f['samples'][...][:,-1,:,0]
    except IndexError: 
        print('no BGS BRIGHT galaxies in %i' % hpix)
        continue
    except KeyError: 
        print('no BGS BRIGHT galaxies in %i' % hpix)
        continue
    except OSError: 
        print('no posteriors for %i' % hpix)
        continue
    except: 
        print('problem reading %i' % hpix)
        continue 
    
    is_hpix = (bgs_bright['healpix'] == hpix)
    print(hpix, np.sum(is_hpix))

    for i in np.arange(len(bgs_bright))[is_hpix]: 
        is_target = (tid == bgs_bright['TARGETID'][i])

        if np.sum(is_target) == 1:
            mstars[i,:] = mcmc[is_target]

9144 4
9145 269
9146 363
9147 640
9148 433
9149 94
9150 695
9151 437
9334 99
9335 69
9337 402
9338 38
9339 703
9340 773
9341 664
9342 570
9343 430
9424 43
9425 580
9427 101
9428 626
9429 518
9430 352
9431 74
9512 179
9514 289
9600 49
9791 1
9812 3
9813 255
9814 402
9815 817
9817 67
9818 91
9819 242
9820 1001
9821 778
9822 740
9823 725
9829 287
9830 127
9831 633
9834 218
9835 442
9836 376
9837 574
9838 315
9839 535
9840 747
9841 562
9842 729
9843 419
9844 698
9845 467
9848 455
9849 145
9850 387
9851 5
9877 324
9878 15
9879 670
9884 49
9885 624
9887 147
9920 587
9921 526
9922 615
9923 446
9924 212
9925 48
9926 81
9928 551
9929 343
9930 188
9931 5
no BGS BRIGHT galaxies in 9932
9983 3
9984 293
no posteriors for 9985
9986 715
9987 25
9992 700
9993 3
9994 345
10016 12
10145 1
10146 53
10147 696
10148 45
10150 809
10151 156
10152 483
10153 1000
10154 787
10155 737
10156 890
10157 238
10158 806
10159 76
10197 47
10198 3
10199 484
10204 148
10205 659
10206 331
10207 621
10228 375
10229 469
102

In [19]:
bgs_bright['logM_mcmc'] = mstars

In [12]:
print("%i galaxies don't have posteriors" % np.sum(mstars[:,0] == -999.))

1341 galaxies don't have posteriors


In [15]:
bgs_bright[:5]

RA,DEC,TARGETID,NTILE,TILES,Z,COMP_TILE,rosette_number,rosette_r,FRACZ_TILELOCID,BITWEIGHTS [2],PROB_OBS,WEIGHT_ZFAIL,WEIGHT,flux_g_dered,flux_r_dered,flux_z_dered,flux_w1_dered,flux_w2_dered,NZ,WEIGHT_FKP,phi,theta,healpix
float64,float64,int64,int64,bytes43,float64,float64,float64,float64,float64,int64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float64,float64,float64,int64
191.20458196461243,60.31535787110577,39633390185481345,1,100,0.08969256109731599,0.64,3.0,1.613220114177946,1.0,-1 .. -1,1.0,1.0,1.0,11.618794,18.39226,20.892622,13.421043,9.27371,0.04778151135868713,0.0030821516772042766,3.3371495001818547,0.5180947424254233,11226
187.04622815471808,60.82670180284912,39633396124617897,1,100,0.37400455377945924,0.64,3.0,1.6801915534232357,1.0,-1 .. -1,1.0,1.0,1.0,4.046807,16.82531,38.79777,85.02852,54.11833,0.0009577827729000268,0.133626516949366,3.264572534736348,0.5091701072064087,11237
187.83760708675698,60.38987543677365,39633393155049303,1,100,0.1731108191825362,0.64,3.0,1.7267282368438097,0.5,6932875859169517021 .. -6858338461939850770,0.4609375,1.0,2.15,13.26433,27.728409,46.020153,37.03337,25.867756,0.01270745015908088,0.01149149450827468,3.2783847027313433,0.5167941655539476,11237
187.05117249536465,62.614542270156655,39633416324383107,1,100,0.2703579310142208,0.64,3.0,1.5579854421783794,1.0,-1 .. -1,1.0,1.0,1.0,4.4575434,17.798513,37.420948,46.075268,30.963612,0.004051015958871262,0.035183206585599834,3.264658829759971,0.47796640455149814,11251
191.14757099005186,60.345815959235,39633390185480938,1,100,0.2819459922463374,0.64,3.0,1.5743490921284766,0.5,4864741233190265570 .. 434972757299228716,0.359375,1.0,2.74468085106383,6.782468,16.732075,29.996437,31.939342,25.343126,0.004051015958871262,0.035183206585599834,3.3361544709660023,0.5175631485037057,11226


In [16]:
bgs_bright['healpix', 'Z'][mstars[:,0] == -999.]

healpix,Z
int64,float64
11237,0.2905843068080386
11237,0.12366938339663507
11407,0.25464967502213787
11402,0.2519833677396139
11427,0.353025817104069
11426,0.34498585576378826
11428,0.25408657571747206
11424,0.2867461360923218
11425,0.28364597849535805
